In [1]:
import numpy as np

In [16]:
a=np.random.rand(10,1)
b=np.random.rand(10,1)

In [17]:
a*b.T

array([[0.64363576, 0.65009754, 0.61756021, 0.18317342, 0.34559324,
        0.15330469, 0.63728903, 0.44204631, 0.41180086, 0.26802663],
       [0.41960297, 0.42381558, 0.40260364, 0.11941554, 0.22530126,
        0.09994333, 0.41546537, 0.28818154, 0.26846374, 0.17473356],
       [0.48608712, 0.49096719, 0.46639432, 0.13833638, 0.2609992 ,
        0.11577889, 0.48129393, 0.33384257, 0.31100058, 0.20241929],
       [0.54542745, 0.55090326, 0.52333059, 0.15522414, 0.29286135,
        0.1299129 , 0.54004912, 0.37459726, 0.34896677, 0.22713014],
       [0.67842602, 0.68523708, 0.65094101, 0.19307444, 0.36427349,
        0.16159123, 0.67173623, 0.46594011, 0.43405981, 0.28251419],
       [0.66429812, 0.67096734, 0.63738547, 0.18905376, 0.35668767,
        0.15822616, 0.65774764, 0.45623713, 0.42502072, 0.27663097],
       [0.32531207, 0.32857804, 0.31213273, 0.09258113, 0.17467278,
        0.07748461, 0.32210424, 0.22342295, 0.20813602, 0.13546838],
       [0.49829375, 0.50329637, 0.4781064

In [16]:
np.inf

inf